## Simulating Points In A Unit Disc

To simulate points in a disc one approach is to simulate points in a unit square and reject points falling outside the disc. But this wastes some points (~22%).

Let's try to directly simulate points in the disc using polar coordinates ($R$, $\Theta$). 

Naive approach is to simulate $R$, $\Theta$ as follows:
$R \sim  U[0,1]$ and $ \Theta \sim U[0, 2\pi]$. This is incorrect because $R$ is not $U \sim [0,1]$

The distribution of $R$ can be computed as follows:

$ P(R < r)  = \frac{\pi r^2}{\pi.1^2} $. Therefore, the CDF is $ F_R(r) = r^2 $

Using the inverse CDF transform we can simulate the random variable $R$ as $\sqrt{U[0,1]}$ which is the correct way to simulate the random variable $R$. 

The visualization compares both the approaches for simulating $R$. Without the square root transform the points are not simulated uniformly within the disc, with concentration of points close to the center.

In [1]:
import numpy as np

import ipywidgets as w

import bqplot.pyplot as plt
from bqplot import *

In [2]:
n_slider = w.IntSlider(description='Points', min=1000, value=2000, max=5000, step=500)
n_slider.layout.width = '400px'

do_sqrt_check_box = widgets.Checkbox(description='Sqrt Transform')

go_btn = w.Button(description='GO', button_style='success')

pi_label_tmpl = '<div style="font-size: 20px; font-weight: bold;">pi = {pi:.4f} for n = {n}</div>'
pi_label = w.HTML(pi_label_tmpl.format(pi=0, n=0))
xs, ys = LinearScale(min=-1, max=1), LinearScale(min=-1, max=1)

# points inside arc
dots = ScatterGL(x=[], 
                 y=[],
                 colors=['limegreen'],
                 scales={'x': xs, 'y': ys},
                 stroke='black',
                 default_size=20)

xax = Axis(scale=xs, grid_lines='none', num_ticks=0)
yax = Axis(scale=ys, orientation='vertical', grid_lines='none', num_ticks=0)
yax.offset = dict(value=0, scale=xs)
xax.offset = dict(value=0, scale=ys)

# circle
x = np.linspace(-1, 1, 500)
y = np.sqrt(1 - x ** 2)
circle = plt.plot(x, [y, -y], 'm', stroke_width=2)

fig = Figure(marks=[dots, circle], axes=[xax, yax], title='Points In A Disc')
fig.layout.width = '500px'
fig.layout.height = '500px'

def animate(*args):
    N = n_slider.value

    # simulate r, theta from two independent uniforms
    do_sqrt = do_sqrt_check_box.value
    u1, u2 = np.random.rand(2, N)
    r = np.sqrt(u1) if do_sqrt else u1
    theta = 2 * np.pi * u2
    
    x_points, y_points = [], []
    
    for i in range(N):
        xi = r[i] * np.cos(theta[i])
        yi = r[i] * np.sin(theta[i])
        
        x_points.append(xi)
        y_points.append(yi)
        
        with dots.hold_sync():
            dots.x, dots.y = x_points, y_points

go_btn.on_click(animate)

w.VBox([w.HBox([n_slider, do_sqrt_check_box, go_btn]), fig])